In [1]:
import os
from os.path import join

import torch
from torch.utils.data import TensorDataset, DataLoader
import scanpy as sc
import numpy as np
import pandas as pd
import anndata
import dask.dataframe as dd
import dask.array as da
import pickle

from scipy.sparse import csr_matrix

In [2]:
import sklearn
print(sklearn.__version__)

1.7.2


In [6]:
# === Load data ===
data_dir = "/projects/b1042/GoyalLab/jaekj/perturb-seq"
z_train = np.load(os.path.join(data_dir, "z_train.npy"))
z_val = np.load(os.path.join(data_dir, "z_val.npy"))
z_test = np.load(os.path.join(data_dir, "z_test.npy")) 

y_train = np.load(os.path.join(data_dir, "y_train.npy")).astype(int)
y_val = np.load(os.path.join(data_dir, "y_val.npy")).astype(int)
y_test = np.load(os.path.join(data_dir, "y_test.npy")).astype(int)

In [7]:
# === Order test data by class (same as your existing code) ===
unique_classes = np.sort(np.unique(y_test))
z_test_ordered = []
y_test_ordered = []

for class_label in unique_classes:
    class_indices = np.where(y_test == class_label)[0]
    z_test_ordered.append(z_test[class_indices])
    y_test_ordered.append(y_test[class_indices])
    print(f"Class {class_label}: {len(class_indices)} samples")

z_test_ordered = np.vstack(z_test_ordered)
y_test_ordered = np.concatenate(y_test_ordered)

Class 0: 339 samples
Class 1: 131 samples
Class 2: 68 samples
Class 3: 117 samples
Class 4: 70 samples
Class 5: 68 samples
Class 6: 80 samples
Class 7: 122 samples
Class 8: 31 samples
Class 9: 86 samples
Class 10: 84 samples
Class 11: 63 samples
Class 12: 326 samples
Class 13: 101 samples
Class 14: 79 samples
Class 15: 106 samples
Class 16: 128 samples
Class 17: 75 samples
Class 18: 91 samples
Class 19: 272 samples
Class 20: 55 samples
Class 21: 56 samples
Class 22: 60 samples
Class 23: 60 samples
Class 24: 54 samples
Class 25: 69 samples
Class 26: 116 samples
Class 27: 48 samples
Class 28: 98 samples
Class 29: 106 samples
Class 30: 78 samples
Class 31: 81 samples
Class 32: 46 samples
Class 33: 81 samples
Class 34: 79 samples
Class 35: 89 samples
Class 36: 81 samples
Class 37: 86 samples
Class 38: 60 samples
Class 39: 332 samples
Class 40: 62 samples
Class 41: 76 samples
Class 42: 112 samples
Class 43: 65 samples
Class 44: 67 samples
Class 45: 62 samples
Class 46: 49 samples
Class 47: 

In [9]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

# === Train Logistic Regression ===
print("\n[Info] Training logistic regression classifier...")

clf = LogisticRegression(
    penalty='l2',          
    C=1.0,                 
    solver='lbfgs',        
    max_iter=1000,
    multi_class='multinomial', 
    random_state=42,
    verbose=1
)

clf.fit(z_train, y_train)


[Info] Training logistic regression classifier...


/projects/b1042/GoyalLab/jaekj/python/DL_py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   59.1s finished


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42,
                   verbose=1)

In [10]:
# === Get predictions ===
posteriors_lr = clf.predict_proba(z_test_ordered)  # shape [N, C]
y_pred_lr = clf.predict(z_test_ordered)             # shape [N]

In [12]:
from sklearn.metrics import accuracy_score, f1_score

# === Evaluate ===
print("\n=== Performance Comparison ===")
print(f"Logistic Regression accuracy: {accuracy_score(y_test_ordered, y_pred_lr):.4f}")
print(f"Logistic Regression macro-F1: {f1_score(y_test_ordered, y_pred_lr, average='macro'):.4f}")


=== Performance Comparison ===
Logistic Regression accuracy: 0.9164
Logistic Regression macro-F1: 0.9326


In [13]:
# === Save model and predictions ===
save_dir = './'

with open(os.path.join(save_dir, 'logistic_regression_clf.pkl'), 'wb') as f:
    pickle.dump(clf, f)

np.save(os.path.join(save_dir, 'y_pred_lr.npy'), y_pred_lr)
np.save(os.path.join(save_dir, 'posteriors_lr.npy'), posteriors_lr)

print(f"\n[Info] Model and predictions saved to {save_dir}")


[Info] Model and predictions saved to ./
